In [48]:
import numpy as np
import pandas as pd

In [49]:
df=pd.read_csv('IMDB Dataset.csv',encoding = "ISO-8859-1")

In [50]:
df.head(5)

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [51]:
#text preproxessing
#sample 10000 rows
#remove html tags
#remove special chars
#convert to lower case
#remove stop words
#stemming

In [52]:
df=df.sample(10000)

In [53]:
from sklearn.preprocessing import LabelEncoder
encoder=LabelEncoder()
df['sentiment']=encoder.fit_transform(df['sentiment'])
df.head(5)

,review,sentiment
42630,Swift's writing really has more in common with...,1
13429,The original story had all the ingredients to ...,0
45777,"Too bad, I really like Kristen Cloke and Gary ...",0
34836,"This movie is phoniness incarnate, a straight ...",0
3974,"A gem from Japan, where so many of the world's...",1


In [54]:
#for removing html tag, this is taken from net copied
import re
clean=re.compile('<.*?>')
re.sub(clean, '', df.iloc[2].review)

'Too bad, I really like Kristen Cloke and Gary Busey. But the director failed to put this together. There\'s a lot of action, a lot of promise, but it all comes off hokey. The director didn\'t do his job. Promising action comes off lame. So much seems contrived in a desperate attempt to save the film. This version of "The Rage" (DirecTV credits it as 1996) simply isn\'t worth the time to watch it. Another director would have done a better job.'

In [55]:
def clean_html(text):
  clean=re.compile('<.*?>')
  return re.sub(clean, '', text)

In [56]:
df['review']=df['review'].apply(clean_html)

In [57]:
df.head(5)

,review,sentiment
42630,Swift's writing really has more in common with...,1
13429,The original story had all the ingredients to ...,0
45777,"Too bad, I really like Kristen Cloke and Gary ...",0
34836,"This movie is phoniness incarnate, a straight ...",0
3974,"A gem from Japan, where so many of the world's...",1


In [61]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')
from nltk.corpus import stopwords
import string
from nltk.stem.porter import PorterStemmer
ps=PorterStemmer()
# making function that do all these tasks


def transform_text(text):
  text=text.lower()
  text=nltk.word_tokenize(text)

  y=list()
  for i in text:
    if i.isalnum():
      y.append(i)

  #text=y  list is mutable datatype donot copy like this
  text=y[:]
  y.clear()

  for i in text:
    if i not in stopwords.words('english') and i not in string.punctuation:
      y.append(i)

  text=y[:]
  y.clear()

  for i in text:
    y.append(ps.stem(i))

  return " ".join(y) # for returning the string of list

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [62]:
df['review']=df['review'].apply(transform_text)

In [63]:
df.head(5)

,review,sentiment
42630,swift write realli common kafka orwel writer c...,1
13429,origin stori ingredi make thoroughli grip film...,0
45777,bad realli like kristen cloke gari busey direc...,0
34836,movi phoni incarn straight 11 10 phoni scale f...,0
3974,gem japan mani world best film made today styl...,1


In [116]:
from sklearn.feature_extraction.text import CountVectorizer
cv= CountVectorizer(max_features=10000) #try taking 1k most frequent features instead of 35k

In [117]:
X=cv.fit_transform(df['review']).toarray()

In [118]:
y=df['sentiment'].values

In [119]:
print(X.shape,y.shape)

(10000, 10000) (10000,)


In [120]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=2) #test size is 20 percent

In [121]:
from sklearn.naive_bayes import GaussianNB, MultinomialNB, BernoulliNB # because we dont know distribution of data
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score

In [122]:
gnb=GaussianNB()
mnb=MultinomialNB()
bnb=BernoulliNB()

In [123]:
gnb.fit(X_train,y_train)
y_predg=gnb.predict(X_test)
print(accuracy_score(y_test,y_predg))
print(confusion_matrix(y_test,y_predg))
print(precision_score(y_test,y_predg))  # accuracy zada matter nahi karta precision krta hai kyuki imbalanced data hai to precision zada chahiye

0.6355
[[787 196]
 [533 484]]
0.711764705882353


In [124]:
mnb.fit(X_train,y_train)
y_predm=mnb.predict(X_test)
print(accuracy_score(y_test,y_predm))
print(confusion_matrix(y_test,y_predm))
print(precision_score(y_test,y_predm))

0.8395
[[845 138]
 [183 834]]
0.8580246913580247


In [125]:
bnb.fit(X_train,y_train)
y_predb=bnb.predict(X_test)
print(accuracy_score(y_test,y_predb))
print(confusion_matrix(y_test,y_predb))
print(precision_score(y_test,y_predb))

0.833
[[845 138]
 [196 821]]
0.8561001042752867


In [115]:
#we can apply loop for max_features in CountVectorizer


In [ ]:
#mnb is working better